In [ ]:
#use niwrap3912 conda environment
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import map_coordinates
import matplotlib
import matplotlib.pyplot as plt

def sample_at_points(volume, points, affine):
    """Sample volume intensities at given points using trilinear interpolation"""
    inv_affine = np.linalg.inv(affine)
    homogeneous = np.column_stack([points, np.ones(len(points))])
    voxel_coords = (inv_affine @ homogeneous.T)[:3, :]
    return map_coordinates(volume.get_fdata(), voxel_coords, order=1, mode='constant', cval=0.0)

def generate_layer_intensity_profile(vol, layer_type, path_surf_norm, path_surf_coords, 
                                     save_path=None, sort_by_ap=True, spacing_mm=0.12,
                                     dist_max_mm=2, clim_max=3, cmap='RdBu_r', do_diff=False, fontsize = 9, dist_method=0):
                                     
    """Generate and plot intensity profiles at varying distances from cortical surface"""
    
    # Load surface data
    surf_norm = nib.load(path_surf_norm)
    surf_coords = nib.load(path_surf_coords)
    
    # Extract coordinates and normals
    norm_xyz = np.array([surf_norm.darrays[i].data for i in range(3)])
    surf_xyz = np.array([surf_coords.darrays[i].data for i in range(3)])

    if dist_method == 0:
        #method 1 
        #calculate half voxel up (spacing mm/2) and down (spacing mm/2) from the surface along the surfarce normal 
        dist_array = np.flipud(np.concatenate([-np.arange(spacing_mm/2, dist_max_mm, spacing_mm)[::-1], 
                                            np.arange(spacing_mm/2, dist_max_mm, spacing_mm)]))
    elif dist_method == 1:
        #method 2 
        #calculate full voxel length along the surface normal
        dist_array = np.flipud(np.concatenate([-np.arange(spacing_mm, dist_max_mm, spacing_mm)[::-1], [0], 
                                            np.arange(spacing_mm, dist_max_mm, spacing_mm)]))
    
    # Generate sampling points at different distances
    all_values = []
    for dist in dist_array:
        sample_points = surf_xyz.T + dist * norm_xyz.T
        values = sample_at_points(vol, sample_points, vol.affine)
        all_values.append(values)
    
    all_values = np.array(all_values)
    
    # Sort by anterior-posterior if requested
    ap_order = None
    if sort_by_ap:
        ap_coords = surf_xyz[1, :]  # Y coordinate for AP ordering
        ap_order = np.argsort(ap_coords)
        all_values = all_values[:, ap_order]
    
    return all_values, dist_array, ap_order

In [ ]:

#for BigBrain
vol_path = '/Users/dennis.jungchildmind.org/Desktop/BigBrain/HistologicalSpaceData/full16_100um_optbal.nii'#use the highest resolution volume 
surface_path = '/Users/dennis.jungchildmind.org/Desktop/BigBrain/PlosBiology2020gii/'

#make these surfnorm and coord files using the calculate_surface_normal_and_coordinates.py script
#or you can just do by workbench command

save_path_subject = '/Users/dennis.jungchildmind.org/Desktop/BigBrain/PlosBiology2020gii/'

hemi = 'lh'
layer_type = 'layer6'
inf_surface_lh = f'surfgii/{layer_type}_left_327680.surf.gii'# layer3 boundary is equivalent to the inf surface of ex vivo data    
inf_surface_rh = f'surfgii/{layer_type}_right_327680.surf.gii'
inf_surfnorm_lh = f'surfnorms/{layer_type}_left_327680.surfnorm.func.gii'
inf_surfnorm_rh = f'surfnorms/{layer_type}_right_327680.surfnorm.func.gii'
inf_coord_lh = f'surfnorms/{layer_type}_left_327680.coord.func.gii'
inf_coord_rh = f'surfnorms/{layer_type}_right_327680.coord.func.gii'

#params for the mainscript
params = {'sort_by_ap': False, 'spacing_mm': 0.12, 'dist_max_mm': 2.0, 'clim_max': 3, 'do_diff':True, 'dist_method':0, 'fontsize': 8}


vol = nib.load(vol_path)
# Check if required surface files exist


if hemi == 'lh':
    surf_norm_path = os.path.join(surface_path, inf_surfnorm_lh)
    surf_coord_path = os.path.join(surface_path, inf_coord_lh)
elif hemi == 'rh':
    surf_norm_path = os.path.join(surface_path, inf_surfnorm_rh)
    surf_coord_path = os.path.join(surface_path, inf_coord_rh)
else:
    print(f"Warning: {hemi} not found, skipping {layer_type}")
    exit()

if not os.path.exists(surf_norm_path):
    print(f"Warning: {surf_norm_path} not found, skipping {layer_type}")

if not os.path.exists(surf_coord_path):
    print(f"Warning: {surf_coord_path} not found, skipping {layer_type}")

    
print(f"Processing layer: {layer_type}")
#calculate intensity profileS
all_values, dist_array, ap_order = generate_layer_intensity_profile(
    vol, layer_type,
    surf_norm_path,
    surf_coord_path,
    save_path_subject, **params
)

#save all_values, dist_array, ap_order as npz 
#np.savez(f'{save_path_subject}/bigbrain_{hemi}_{layer_type}_{int(params["spacing_mm"]*1000)}um_method{int(params["dist_method"])}_manual_raw_intensity.npz', all_values=all_values, dist_array=dist_array, ap_order=ap_order)

In [ ]:
#PLOT ALL_VALUES using functions from the context
import matplotlib.pyplot as plt
import numpy as np

# Configuration for plotting
fontsize = 8
percentil_low = 20
percentil_high = 80
cmap = 'plasma'
data_types = ['raw', 'raw_zscore', 'diff', 'diff_zscore']

def load_and_process_data_from_arrays(all_values, dist_array, sorted_order=None):
    """Process data arrays and compute processed versions (diff, zscore) with NaN handling."""
    
    # Use provided sorted order or compute new one
    if sorted_order is None:
        # Use nanmean for sorting to handle NaN values
        sorted_order = np.argsort(np.nanmean(all_values, axis=0))
    
    # Extract and sort raw values
    raw = all_values[:, sorted_order]
    
    # Compute processed versions with NaN handling
    # For zscore, use nanmean and nanstd
    raw_mean = np.nanmean(raw, axis=0, keepdims=True)
    raw_std = np.nanstd(raw, axis=0, keepdims=True)
    raw_std[raw_std == 0] = 1  # Avoid division by zero
    raw_zscore = (raw - raw_mean) / raw_std
    
    # For diff, use np.diff which preserves NaN values
    diff = np.diff(raw, axis=0)
    diff_mean = np.nanmean(diff, axis=0, keepdims=True)
    diff_std = np.nanstd(diff, axis=0, keepdims=True)
    diff_std[diff_std == 0] = 1  # Avoid division by zero
    diff_zscore = (diff - diff_mean) / diff_std
    
    return {
        'raw': raw,
        'raw_zscore': raw_zscore,
        'diff': diff,
        'diff_zscore': diff_zscore,
        'sorted_order': sorted_order
    }

def set_colorbar_limits(data, data_type):
    """Set appropriate colorbar limits based on data type with NaN handling."""
    # Use nanpercentile to handle NaN values
    if data_type == 'raw':
        return np.nanpercentile(data, percentil_low), np.nanpercentile(data, percentil_high)
    elif data_type in ['diff','diff_zscore','raw_zscore']:
        p_low, p_high = np.nanpercentile(data, percentil_low), np.nanpercentile(data, percentil_high)
        return p_low, p_high

def setup_colorbar(im, ax, data_type, vmin, vmax):
    """Setup colorbar with appropriate ticks and labels."""
    cbar = plt.colorbar(im, ax=ax)
    
    if data_type in ['diff', 'diff_zscore']:
        cbar.set_ticks([vmin, 0, vmax])
        cbar.set_ticklabels([f'{vmin:.2f}', '0', f'{vmax:.2f}'])
    
    return cbar

def create_subplot(ax, data, title, data_type, plot_dist_values):
    """Create a single subplot with proper formatting and NaN handling."""
    # Create masked array to handle NaN values in visualization
    masked_data = np.ma.masked_invalid(data)
    #remove 65536 from the data
    masked_data = np.delete(masked_data, 65535, axis=1)
    im = ax.imshow(masked_data, aspect='auto', 
                   extent=[0, data.shape[1], plot_dist_values[-1], plot_dist_values[0]], 
                   cmap=cmap)
    
    vmin, vmax = set_colorbar_limits(data, data_type)
    
    # Handle case where vmin or vmax might be NaN
    if np.isnan(vmin) or np.isnan(vmax):
        print(f"Warning: NaN values in colorbar limits for {title}, using data min/max")
        vmin, vmax = np.nanmin(data), np.nanmax(data)

    im.set_clim(vmin, vmax)
    
    setup_colorbar(im, ax, data_type, vmin, vmax)
    
    ax.set_xlabel('Vertices (ordered by column mean)', fontsize=fontsize+1)
    ax.set_ylabel('Rel. Inf Surf (mm)', fontsize=fontsize)
    ax.set_title(title, fontsize=fontsize+1, fontweight='bold')
    
    return im

# Process the data
processed_data = load_and_process_data_from_arrays(all_values, dist_array)

# Create combined 2x2 figure
combined_fig, axes = plt.subplots(2, 2, figsize=(12, 4))
axes = axes.flatten()

for i, data_type in enumerate(data_types):
    data = processed_data[data_type]
    create_subplot(axes[i], data, f'{data_type}', data_type, dist_array)

plt.suptitle(f'BigBrain {hemi} {layer_type} - All Data Types', fontsize=fontsize+2, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
print(dist_array)

In [ ]:
#get dist value average of every two values 
dist_array_between = (dist_array[1:] + dist_array[:-1]) / 2
print(dist_array_between)

data_type='raw'
data = processed_data[data_type]

if data_type == 'raw':
    xval = dist_array
elif data_type ==  'zscore':
    xval = dist_array_between

# Create continuous errorbar using fill_between
mean_values = np.median(data, axis=1)
std_values = np.std(data, axis=1)
sem_values = std_values / np.sqrt(data.shape[1]-1)

plt.plot(xval, mean_values, 'b-', linewidth=2)
plt.fill_between(xval, mean_values - std_values, mean_values + std_values, 
                 alpha=0.3, color='blue')
                 
